In [29]:
import cv2
import lzma
import numpy as np
import matplotlib.pyplot as plt

dst_name = 'panic.png'
image = cv2.imread('dont_panic.png')

In [28]:
def toBits(data):
    if type(data) == str:
        return ''.join([f'{ord(i):08b}' for i in data])
    elif type(data) == bytes or type(data) == np.ndarray:
        return [f'{i:08b}' for i in data]
    elif type(data) == int or type(data) == np.uint8:
        return f'{data:08b}'
    raise TypeError('Data type not supported!')

In [27]:
def getData(image):
    out = ''
    for values in image:
        for pixel in values:
            b,g,r = toBits(pixel)
            out += b[-1]
            out += g[-1]
            out += r[-1]
    out = [int(out[i:i+8],2) for i in range(0,len(out),8)]
    return ''.join([chr(i) for i in out])

In [26]:
def hideData(image, secret: bytes):
    assert type(secret) == bytes
    secret_bits = ''.join(toBits(secret))
    ###
    h, w, channels = image.shape
    pixels = h * w
    max_bit_len = pixels * channels
    
    bit_index = 0
    bit_len = len(secret_bits)
    
    if bit_len > max_bit_len:
        raise RuntimeError('Not enough bits to hide data')
    print(f'Available:\t[{max_bit_len} bits]\t[{max_bit_len//8} chars]')
    print(f'Payload:\t[{bit_len} bits]\t[{len(secret)} chars]')
    
    for values in image:
        for pixel in values:
            b,g,r = toBits(pixel)
            assert len(b) == len(g) == len(r)
            if bit_index < bit_len:
                pixel[0] = int(b[:-1] + secret_bits[bit_index],2)
                bit_index += 1
            if bit_index < bit_len:
                pixel[1] = int(g[:-1] + secret_bits[bit_index],2)
                bit_index += 1
            if bit_index < bit_len:
                pixel[2] = int(r[:-1] + secret_bits[bit_index],2)
                bit_index += 1
        if bit_index == bit_len:
            break